## Evaluating full test set performance

In [1]:
import torch
from torch.autograd import Variable
from capsnet import CapsNet
import utils

USE_GPU = True

### Load model and weights from past training

In [2]:
model = CapsNet(n_conv_channel=256,
                n_primary_caps=8,
                primary_cap_size=1152,
                output_unit_size=16,
                n_routing_iter=3)

In [3]:
def test(epoch, model, dataloader):
    accs = []
    losses = []
    model.eval()

    for i, (X, y) in enumerate(dataloader):
        target = utils.one_hot(y, model.final_caps.n_unit)

        X, target = Variable(X), Variable(target)
        if USE_GPU:
            X, target = X.cuda(), target.cuda()

        y_hat = model(X)
        loss = model.loss(y_hat, target)

        preds = model.capsule_prediction(y_hat)
        acc = utils.categorical_accuracy(y.float(), preds.cpu().data)
        
        accs.append(acc)
        losses.append(loss.data[0])
    
    return accs, losses

### Test on MNIST

In [14]:
model.load_state_dict(torch.load('./checkpoints/mnist_capsnet_50_epoch.pth'))
model.eval()
model = model.cuda() if USE_GPU else model

trainloader, testloader = utils.mnist_dataloaders('./data',
                                                  128,
                                                  USE_GPU)

In [15]:
accs, losses = test(1, model, testloader)

In [16]:
accs, losses = torch.Tensor(accs), torch.Tensor(losses)

### Average accuracy and loss over the entire test set

In [17]:
accs.mean()

0.10799050632911393

In [12]:
print('Accuracy: {:.3f} Loss: {:.5f}'.format(100*accs.mean(), losses.mean()))

Accuracy: 10.740 Loss: 1.48257


### Test on Fashion MNIST

In [18]:
model = CapsNet(n_conv_channel=256,
                n_primary_caps=8,
                primary_cap_size=1152,
                output_unit_size=16,
                n_routing_iter=3)
model.load_state_dict(torch.load('./checkpoints/fmnist_capsnet_50_epoch.pth'))
model.eval()
model = model.cuda() if USE_GPU else model

trainloader, testloader = utils.mnist_dataloaders('/home/erikreppel/data/fashion-mnist/',
                                                  128,
                                                  USE_GPU)

In [19]:
accs, losses = test(1, model, testloader)
accs, losses = torch.Tensor(accs), torch.Tensor(losses)
print('Accuracy: {:.3f} Loss: {:.5f}'.format(100*accs.mean(), losses.mean()))

Accuracy: 88.904 Loss: 0.16416
